In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/feedback-prize-english-language-learning/sample_submission.csv
/kaggle/input/feedback-prize-english-language-learning/train.csv
/kaggle/input/feedback-prize-english-language-learning/test.csv


In [2]:
!pip install word2number

  Preparing metadata (setup.py) ... - \ done
  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5583 sha256=11fcde1ef84d5c3a4b96ace4c06f7b9827a6b1da60163be46a7391cd2a9268b6
  Stored in directory: /root/.cache/pip/wheels/84/ff/26/d3cfbd971e96c5aa3737ecfced81628830d7359b55fbb8ca3b
Successfully built word2number


In [3]:
import nltk
import datetime as dt
import seaborn as sns

# For Text pre-processing
from word2number import w2n
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag, pos_tag_sents

# TF
import tensorflow_hub as hub
from keras.models import Sequential
from keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Embedding, Input, dot

# Sklearn
# Performance metrics
from sklearn.metrics import f1_score 
from sklearn.metrics import accuracy_score
# Binary Relevance
from sklearn.multiclass import OneVsRestClassifier 
# Model
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [4]:
#!pip3 show tensorflow

In [5]:
train_df = pd.read_csv('/kaggle/input/feedback-prize-english-language-learning/train.csv')
test_df = pd.read_csv('/kaggle/input/feedback-prize-english-language-learning/test.csv')

In [6]:
train_df.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


In [7]:
train_df.describe()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
count,3911.000000,3911.000000,3911.000000,3911.000000,3911.000000,3911.000000
mean,3.127077,3.028254,3.235745,3.116850,3.032856,3.081053
std,0.662542,0.644399,0.583148,0.655997,0.699841,0.671450
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,2.500000,2.500000,3.000000,2.500000,2.500000,2.500000
50%,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
75%,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [8]:
train_df.isnull().sum()

text_id        0
full_text      0
cohesion       0
syntax         0
vocabulary     0
phraseology    0
grammar        0
conventions    0
dtype: int64

In [9]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [11]:
# remove nums from sentences
def nums(x):
    words = []
    remove = []
    for i in x:
        try:
            w2n.word_to_num(i)
            remove.append(i)
        except:
            words.append(i)
    
    return "".join(words)

In [12]:
import re
import string
def clean_p1(sen):
    sen = str(sen).lower()
    sen = re.sub('\[.*?\]', '', sen)
    sen = re.sub('https?://\S+|www\.\S+', '', sen)
    sen = re.sub('<.*?>+', '', sen)
    sen = re.sub('[%s]' % re.escape(string.punctuation), '', sen)
    sen = re.sub('\n', '', sen)
    sen = re.sub('\w*\d\w*', '', sen)
    return sen

In [13]:
train_df['full_text'] = train_df['full_text'].apply(clean_p1)
train_df['full_text'] = train_df['full_text'].map(lambda x: nums(x))
train_df.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,i think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,when a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,dear principalif u change the school policy of...,3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,the best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


In [14]:
test_df['full_text'] = test_df['full_text'].apply(clean_p1)
test_df['full_text'] = test_df['full_text'].map(lambda x: nums(x))
test_df.head()

,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,do you think students would benefit from being...
2,00367BB2546B,thomas jefferson once states that it is wonder...


In [15]:
# cleaning the text
def textPre(sen):
    #tokenize
    tokens = word_tokenize(sen)
    stop_words = set(stopwords.words('english'))
    
    #remove
    filtered_words = [w for w in tokens if w not in stop_words]
    
    #stemming
    #stem = PorterStemmer()
    #stem_words = [stem.stem(w) for w in filtered_words]
    
    #lemmatize
    lemma = WordNetLemmatizer()
    lemma_words = [lemma.lemmatize(w) for w in filtered_words]
    
    return " ".join(lemma_words)

In [16]:
train_df['full_text'] = train_df['full_text'].apply(textPre)
train_df.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,think student would benefit learning homebecau...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,problem change let best matter happening chang...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,dear principalif u change school policy grade ...,3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,best time life become agree greatest accomplis...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,small act kindness impact people change people...,2.5,3.0,3.0,3.0,2.5,2.5


In [17]:
test_df['full_text'] = test_df['full_text'].apply(textPre)
test_df.head()

,text_id,full_text
0,0000C359D63E,person experience job always going good people...
1,000BAD50D026,think student would benefit able attend class ...
2,00367BB2546B,thomas jefferson state wonderful much get done...


**Split**

In [18]:
X_train, X_val, Y_train, Y_val = train_test_split(train_df['full_text'],
                                                  train_df[['cohesion', 'syntax', 'vocabulary',
                                                              'phraseology', 'grammar', 'conventions']],
                                                  test_size=0.25,
                                                  random_state=0)

**Encoder**

In [19]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [20]:
X_train.shape

(2933,)

In [21]:
X_val.shape

(978,)

In [22]:
train_embed = embed(X_train)
train_embed

<tf.Tensor: shape=(2933, 512), dtype=float32, numpy=
array([[-0.00601561, -0.06652603,  0.02108635, ..., -0.05108357,
        -0.0123414 , -0.04239651],
       [-0.02461828, -0.06602024,  0.05491249, ..., -0.04495463,
        -0.03307585, -0.03122788],
       [ 0.02077105, -0.05657504,  0.04882124, ..., -0.01431783,
        -0.05615058, -0.03726263],
       ...,
       [-0.00609461, -0.05737458,  0.01522228, ..., -0.05757954,
        -0.04798666, -0.02569435],
       [-0.04165009, -0.01235889,  0.05236395, ..., -0.05865194,
        -0.04272122, -0.03269024],
       [ 0.05272863, -0.06375311, -0.03791047, ..., -0.00301023,
        -0.0579295 ,  0.00099267]], dtype=float32)>

In [23]:
val_embed = embed(X_val)
val_embed

<tf.Tensor: shape=(978, 512), dtype=float32, numpy=
array([[ 0.03024355, -0.05718406,  0.05548753, ..., -0.0302106 ,
        -0.05388178,  0.04762599],
       [-0.01791018, -0.06194515,  0.05407249, ...,  0.03332476,
        -0.05954666, -0.0616127 ],
       [-0.01453929, -0.05727329,  0.04859252, ..., -0.04892482,
        -0.05722526,  0.04050262],
       ...,
       [ 0.02622108, -0.06949209,  0.04752782, ..., -0.04367892,
         0.02593143,  0.02175921],
       [ 0.01166182, -0.06007691,  0.0328596 , ..., -0.04817416,
        -0.05720191, -0.05212624],
       [-0.03688528, -0.05104927,  0.04550692, ...,  0.0484309 ,
        -0.05076615,  0.01340574]], dtype=float32)>

In [24]:
input_dim = len(train_embed[0])
inputs = Input(shape=(input_dim,))

model=Sequential()
model.add(Embedding(input_dim=512,output_dim=128,input_length=input_dim))
model.add(LSTM(128,return_sequences=True))

model.add(Dense(input_dim,activation='softmax'))
model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(6, activation='linear'))
model.compile(loss='mse',optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 512, 128)          65536     
                                                                 
 lstm (LSTM)                 (None, 512, 128)          131584    
                                                                 
 dense (Dense)               (None, 512, 512)          66048     
                                                                 
 lstm_1 (LSTM)               (None, 128)               328192    
                                                                 
 dense_1 (Dense)             (None, 6)                 774       
                                                                 
Total params: 592,134
Trainable params: 592,134
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
batch_size=128
model.fit(train_embed, Y_train, epochs=5, batch_size=batch_size, verbose=1,
         validation_data=(np.array(val_embed), np.array(Y_val))
         )

Epoch 1/5
23/23 [==============================] - 74s 3s/step - loss: 4.0973 - accuracy: 0.2223 - val_loss: 0.5320 - val_accuracy: 0.2444
Epoch 2/5
23/23 [==============================] - 66s 3s/step - loss: 0.4711 - accuracy: 0.1221 - val_loss: 0.4638 - val_accuracy: 0.0665
Epoch 3/5
23/23 [==============================] - 67s 3s/step - loss: 0.4319 - accuracy: 0.1848 - val_loss: 0.4433 - val_accuracy: 0.2444
Epoch 4/5
23/23 [==============================] - 65s 3s/step - loss: 0.4274 - accuracy: 0.2243 - val_loss: 0.4482 - val_accuracy: 0.2444
Epoch 5/5
23/23 [==============================] - 66s 3s/step - loss: 0.4260 - accuracy: 0.2182 - val_loss: 0.4413 - val_accuracy: 0.2444


In [26]:
test_df.head()

,text_id,full_text
0,0000C359D63E,person experience job always going good people...
1,000BAD50D026,think student would benefit able attend class ...
2,00367BB2546B,thomas jefferson state wonderful much get done...


In [27]:
test_df.shape

(3, 2)

In [28]:
test_embed = embed(test_df['full_text'])
test_embed

<tf.Tensor: shape=(3, 512), dtype=float32, numpy=
array([[ 0.03993586, -0.05565797,  0.03880183, ..., -0.04728298,
        -0.05227224, -0.05012098],
       [ 0.00345374, -0.05973887, -0.01852854, ...,  0.0113207 ,
        -0.05845341,  0.04555184],
       [ 0.03767198, -0.06405722,  0.01347981, ...,  0.04687415,
        -0.05737387, -0.02440244]], dtype=float32)>

In [29]:
preds = model.predict(test_embed)

1/1 [==============================] - 1s 773ms/step


In [30]:
preds

array([[3.1100385, 3.070659 , 3.2566898, 3.0985339, 3.0500948, 3.0813792],
       [3.1100385, 3.070659 , 3.2566898, 3.0985339, 3.0500948, 3.0813792],
       [3.1100385, 3.070659 , 3.2566898, 3.0985339, 3.0500948, 3.0813792]],
      dtype=float32)

In [31]:
sub_df = pd.DataFrame(preds, columns=['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions'])

In [32]:
sub_df.insert(0, 'text_id', test_df['text_id'])

In [33]:
sub_df

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,3.110039,3.070659,3.25669,3.098534,3.050095,3.081379
1,000BAD50D026,3.110039,3.070659,3.25669,3.098534,3.050095,3.081379
2,00367BB2546B,3.110039,3.070659,3.25669,3.098534,3.050095,3.081379


In [34]:
sub_df.to_csv('submission.csv', index=False)